# Fetching Weather Data and Uploading to AWS S3

This guide provides steps for fetching weather data using the OpenWeatherMap API and then uploading the data to an Amazon S3 bucket using Python.

## Step 1: Obtain an API Key from OpenWeatherMap

1. Register on the [OpenWeatherMap website](https://openweathermap.org/) and create an account.
2. Find and copy the API key from your account dashboard.

## Step 2: Write Python Function to Fetch Weather Data

In [1]:
import requests

def get_weather_data(city, api_key):
    base_url = "https://api.openweathermap.org/data/2.5/weather"
    params = {'q': city, 'appid': api_key}
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Failed to fetch weather data")

## Step 3: Set Up AWS Credentials for S3 Access
* Install Boto3 using pip install boto3.
* Configure AWS credentials (AWS Access Key ID and Secret Access Key).

In [2]:
!pip install boto3

## Step 4: Write Python Function to Upload Data to S3


In [3]:
import boto3
import json
import aws_access_key

def upload_to_s3(bucket_name, file_name, data):
    s3 = boto3.client('s3',
                      aws_access_key_id=aws_access_key.aws_access_key_id,
                      aws_secret_access_key=aws_access_key.aws_secret_access_key,
                      region_name='us-east-1')

    s3.put_object(Bucket=bucket_name, Key=file_name, Body=json.dumps(data))

## Step 5: Combine the Functions in a Script


In [4]:
# Main execution
api_key = "8b92a0a1e22d01d459c6fd52d03d721e" 
city = "London"
bucket_name = "lab-03-group2"  
file_name = "weather_data.json"

try:
    weather_data = get_weather_data(city, api_key)
    upload_to_s3(bucket_name, file_name, weather_data)
    print("Data uploaded successfully to S3")
except Exception as e:
    print(f"An error occurred: {e}")

Data uploaded successfully to S3


In [5]:
# Example usage
api_key = "8b92a0a1e22d01d459c6fd52d03d721e"  # Replace with your actual API key
city = "London"
try:
    weather_data = get_weather_data(city, api_key)
    print(weather_data)
except Exception as e:
    print(f"An error occurred: {e}")

{'coord': {'lon': -0.1257, 'lat': 51.5085}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 287.08, 'feels_like': 286.72, 'temp_min': 285.75, 'temp_max': 288.03, 'pressure': 1006, 'humidity': 84}, 'visibility': 10000, 'wind': {'speed': 3.13, 'deg': 227, 'gust': 5.36}, 'clouds': {'all': 100}, 'dt': 1712359038, 'sys': {'type': 2, 'id': 2075535, 'country': 'GB', 'sunrise': 1712381017, 'sunset': 1712428896}, 'timezone': 3600, 'id': 2643743, 'name': 'London', 'cod': 200}


# Assignment: Groups to Convert JSON to CSV

Step 1. Take the JSON output and convert it to a Dataframe using pandas
Step 2. Now upload the CSV file to the 'lab-03' S3 bucket in the cloud with the following naming convention: <your group name>_weather_date_london_<datetimestamp>.csv


In [7]:
###### INSERT CODE BELOW ####
import pandas as pd
from io import StringIO


def flatten_json(json_obj, prefix=''):
    """
    Flatten a nested JSON object into a single-level dictionary.

    Args:
        json_obj (dict): The JSON object to be flattened.
        prefix (str, optional): Prefix to be added to keys during flattening. Defaults to ''.

    Returns:
        dict: The flattened dictionary.
    """
    # Initialize an empty dictionary to store flattened key-value pairs
    flat_dict = {}

    # Iterate through key-value pairs in the JSON object
    for key, value in json_obj.items():
        # If the value is another dictionary, recursively flatten it
        if isinstance(value, dict):
            flat_dict.update(flatten_json(value, prefix + key + '_'))  # Recursively call flatten_json with updated prefix
        # If the value is a list, iterate through its elements and recursively flatten each
        elif isinstance(value, list):
            for i, item in enumerate(value):
                flat_dict.update(flatten_json(item, prefix + key + '_'))  # Recursively call flatten_json with updated prefix
        # If the value is a simple value (not a dictionary or list), add it to the flattened dictionary
        else:
            flat_dict[prefix + key] = value
    
    # Return the flattened dictionary
    return flat_dict


try:
    # Convert JSON to DataFrame
    flattened_data = flatten_json(weather_data)
    weather_df = pd.DataFrame([flattened_data])

    # Convert DataFrame to CSV string
    csv_buffer = StringIO()
    weather_df.to_csv(csv_buffer, index=False)

    # Upload the CSV data to the specified S3 bucket with the specified key
    upload_to_s3('lab-03-group2', 'weather_date_london.csv', csv_buffer.getvalue())
    print("Data uploaded successfully to S3")

except Exception as e:
    print(f"An error occurred: {e}")

### END CODE ###

Data uploaded successfully to S3
